In [36]:
import requests
import json
import pandas as pd
from datetime import datetime

# Set query (which uses text input to specify ETH wallet addresses)
query = '''query {
  swaps(first:500 orderBy: timestamp, orderDirection: desc){
    id
    timestamp
    pool{
      token0{
        symbol
      }
      token1{
        symbol
      }
    }
    amount0
    amount1
    amountUSD
  }
}
''' 
#% address

# Point to correct subgraph URL
url = 'https://api.thegraph.com/subgraphs/name/benesjan/uniswap-v3-subgraph'
# Make the request
r = requests.post(url, json={'query': query})
#print(r.status_code)
#print(r.text)

# JSON adjustment
json_data = json.loads(r.text)

# extract JSON to convert to a dataframe
df_data = json_data['data']['swaps']
# convert to dataframe
df1 = pd.DataFrame(df_data)  

#print(df1)
#EDA
#df1.to_excel(r'C:\Users\Nehal.Jain\Documents\Nehal_Personal\Project\Blockchain\Dataset.xlsx', sheet_name='Data', index = False)

#Removing NULLS - Data Cleaning
df = df1.dropna()
#print(df)

#print(df.isnull().values.any())
#print(df.isnull().sum().sum())

#Converting from Unix timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Splitting 'pool' column into 'From_Currency' and 'To_Currency'

df[['From_Currency','To_Currency']] = df.pool.apply(
   lambda x: pd.Series(str(x).split(",")))

#Removing extra data from columns From_Currency and To_Currency : Cleaning
df['From_Currency'] = df['From_Currency'].map(lambda x: str(x)[23:-2])
df['To_Currency'] = df['To_Currency'].map(lambda y: str(y)[23:-3])
#print(df)

#Dropping the pool column
df = df.drop(['pool'], axis=1)

#Cleaning and re-ordering of data
df = df.rename(columns={'amount0': 'From_Currency_Amount', 'amount1': 'To_Currency_Amount', 'amountUSD': 'Amount_USD', 'id': 'Wallet_Address','timestamp': 'Date_Time'})
df = df [['Wallet_Address','From_Currency_Amount','From_Currency','To_Currency_Amount','To_Currency','Amount_USD','Date_Time']]

